In [10]:
import datetime
import re
# import sys
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import hamming_loss, make_scorer, accuracy_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.svm import SVR
from sklearn.metrics import precision_recall_fscore_support as score

In [11]:
def tweet_to_words(raw_tweet):
    tweet = raw_tweet.lower()
    
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', '', tweet)
    
    tweet = re.sub(r'@[\S]+', '', tweet)
    
    tweet = re.sub(r'#(\S+)', r' \1 ', tweet)
    
    tweet = tweet.strip(' "\'')
    
    tweet = re.sub("[^a-zA-Z]", " ", tweet)
    
    tweet = tweet.strip('\'"?!,.():;')
    
    word_tokens = tweet.split()
    
    stop_words = set(stopwords.words('english'))

    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    
    return " ".join(filtered_sentence)

In [12]:
def get_train_reviews(df_):
    num = len(df_)
    train_reviews = []
    for i in range(num):
        train_reviews.append(tweet_to_words(df_['Text'][i]))
    return train_reviews


In [13]:
def get_train_data_features(train_reviews):
    vectorizer = CountVectorizer(analyzer = "word",   \
                                 tokenizer = None,    \
                                 preprocessor = None, \
                                 stop_words = None,   \
                                 max_features = 5000) 
    train_data_features = vectorizer.fit_transform(train_reviews)

    train_data_features = train_data_features.toarray()

    return train_data_features

In [14]:
# trainset
df = pd.read_csv('Tweets_sample.csv', sep = ',')
train_reviews = get_train_reviews(df)
train_data_features = get_train_data_features(train_reviews)

In [15]:
# Multinomial Naive Bayes Model

X_train = train_data_features
y_train = df['Sentiment'].values
model_M = MultinomialNB()
model_M.fit(X_train,y_train)
y_train_M = model_M.predict(X_train)

precision_tr,recall_tr,fscore_tr,suppor_tr=score(y_train,y_train_M,average='macro')
# precision_te,recall_te,fscore_te,support=score(y_test,y_test_M,average='macro')

In [16]:
precision_tr

0.8644715080379428

In [23]:
stocks = ['aapl', 'goog', 'amzn', 'tsla', 'msft', 'nflx']
# stocks = ['aapl']

In [26]:
for stock in stocks:
    dataframe = pd.read_csv('2020/' + stock + '_tweets.csv')
    dataframe.columns = ['Time', 'Text']
    test_reviews = get_train_reviews(dataframe)
    test_data_features = get_train_data_features(test_reviews)
    predict_test = model_M.predict(test_data_features)
    dataframe['sentiment'] = predict_test
    dataframe.to_csv('2021/' + stock + '_2020_sentiment.csv')
        